# RNN

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import division
from __future__ import print_function

from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.utils import plot_model

from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterSampler
from sklearn.decomposition import PCA


from lib import utils
from lib import cols
from lib import rnn_utils
from lib import preprocess
np.random.seed(seed=13)

Using TensorFlow backend.


## Preprocessing

In [3]:
ds_sj, ds_iq = preprocess.preprocess()

lib/utils.py:184: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train_iq = df_dev_iq[is_train_iq]
lib/utils.py:185: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_valid_iq = df_dev_iq[is_valid_iq]
lib/utils.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_devtest_iq = df_dev_iq[is_devtest_iq]


sj train: 675 lines	 valid: 156 lines	 devtest: 105 lines	 test: 260 lines
iq train: 363 lines	 valid: 104 lines	 devtest: 53 lines	 test: 156 lines
sj valid: 0.19%
iq valid: 0.22%


In [4]:
# define some feature names
col_orig_feats = cols.orig_feats
col_scaled_feats = cols.get_scaled_feats(col_orig_feats)

In [5]:
def lag_col(i, cols=col_orig_feats):
    """Get time lag column names
    """
    return ["{0}_lag{1}".format(col, i) for col in cols]

def make_rnn_x(df, ts=5, cols=col_orig_feats):
    """Make RNN's input data
    This makes input data for RNN model.
    This assumes that the given dataframe have columns the name matches `{col}_log{i}`

    Returns:
      X: numpy array with shape (D, T, N)
         where D:sample num, T: time sequence, N: feature num.
    """
    ts_list = [df[lag_col(i, cols)].values for i in range(ts, 0, -1)]
    return np.transpose(ts_list, (1, 0, 2))

def get_X_y(df, ts_offset, ts_len, cols=col_orig_feats ):
    """Make X and y values for RNN    
    """
    X = make_rnn_x(df[ts_offset:], ts_len, cols)
    y = df.total_cases.values[ts_offset:].reshape(-1, 1)
    return X, y

## Create input data

In [6]:
ts_len = 24
n_feats = len(col_scaled_feats)

X_sj, y_sj = get_X_y(ds_sj.df_train, ts_len, ts_len, col_scaled_feats)
X_valid_sj, y_valid_sj = get_X_y(ds_sj.df_valid, 0, ts_len, col_scaled_feats)
X_devtest_sj, y_devtest_sj = get_X_y(ds_sj.df_devtest, 0, ts_len, col_scaled_feats)
X_all_sj, y_all_sj = get_X_y(ds_sj.df_all, ts_len, ts_len, col_scaled_feats)
X_test_sj = make_rnn_x(ds_sj.df_test, ts_len, col_scaled_feats)

ts_len = 24
X_iq, y_iq = get_X_y(ds_iq.df_train, ts_len, ts_len, col_scaled_feats)
X_valid_iq, y_valid_iq = get_X_y(ds_iq.df_valid, 0, ts_len, col_scaled_feats)
X_devtest_iq, y_devtest_iq = get_X_y(ds_iq.df_devtest, 0, ts_len, col_scaled_feats)
X_all_iq, y_all_iq = get_X_y(ds_iq.df_all, ts_len, ts_len, col_scaled_feats)
X_test_iq = make_rnn_x(ds_iq.df_test, ts_len, col_scaled_feats)


In [ ]:
def train(X, y, X_val, y_val, X_devtest, y_devtest, params):
    
    np.random.seed(33) # for reproducibility
    
    from keras.models import Sequential  
    from keras.layers.core import Dense, Activation  
    from keras.layers.recurrent import LSTM
    from keras.callbacks import EarlyStopping
    from keras.optimizers import Adam

    _, ts_len, n = X.shape
    
    hidden_units_1 = params["hidden_units_1"] 
    hidden_units_2 = params["hidden_units_2"] 
    fc_units = params["fc_units"]
    
    lr = params["lr"]
    bs = params["batch_size"]
    patience = params["early_stopping_patience"]
        
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
    
    output_units = 1
    
    model = Sequential()
    model.add(LSTM(hidden_units_1, batch_input_shape=(None, ts_len, n), return_sequences=True))
    model.add(LSTM(hidden_units_2, return_sequences=False))
    model.add(Dense(fc_units))
    model.add(Activation("relu"))  
    model.add(Dense(output_units))
    
    opt = Adam(lr=lr)
    model.compile(loss="mean_absolute_error", optimizer=opt)
    model.fit(X, y, batch_size=bs, nb_epoch=300, validation_data=(X_val, y_val),
                 callbacks=[early_stopping],
                 verbose=2)
    
    score_devtest = get_scores(model, X_devtest, y_devtest)
    score_valid = get_scores(model, X_val, y_val)
    print("valid_score:{0}, devtest_score:{1}".format(score_valid, score_devtest))
    
    return model

def get_scores(model, X, y):
    """Calculate prediction scores
    """
    y_pred = np.around(model.predict(X).squeeze()).astype(int)
    y_true = y.squeeze()
    score = mean_absolute_error(y_true, y_pred)
    return score

def plot_result(model, X, df):
    """Plot prediction and grand truthe
    """
    predicted = model.predict(X)
    df["predict"] = np.r_[np.zeros(ts_len), predicted.squeeze()]
    ax = df.plot(x="week_start_date", y="total_cases", figsize=(15, 5))
    df.plot(x="week_start_date", y="predict", ax=ax)

## Training model

In [ ]:
params = dict(
    hidden_units_1 = 64,
    hidden_units_2 = 48,
    fc_units = 16,
    lr = 0.0003,
    batch_size = 64,
    early_stopping_patience = 5,
)

model_sj = train(X_sj,
                 y_sj,
                 X_valid_sj,
                 y_valid_sj,
                 X_devtest_sj,
                 y_devtest_sj,
                 params)

/home/akiyuki/.pyenv/versions/miniconda2-4.1.11/envs/mle-capstone/lib/python2.7/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


In [ ]:
# save structore
plot_model(model_sj, "./img/keras_model_sj.png")

In [ ]:
params = dict(
    hidden_units_1 = 64,
    hidden_units_2 = 48,
    fc_units = 16,
    lr = 0.0003,
    batch_size = 64,
    early_stopping_patience = 5,
)

model_iq = train(X_iq, y_iq,
                 X_valid_iq,
                 y_valid_iq,
                 X_devtest_iq,
                 y_devtest_iq,
                 params)

## Plot Results

In [ ]:
plot_result(model_sj, X_all_sj, ds_sj.df_all)
plot_result(model_iq, X_all_iq, ds_iq.df_all)

## Save Result

In [ ]:
rnn_utils.save_result(model_sj, model_iq, col_scaled_feats, col_scaled_feats, ds_sj.df_test, ds_iq.df_test, "LSTM")